# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,westport,41.1415,-73.3579,1.02,72,0,2.06,US,1701927765
1,1,mejit,10.2753,170.8646,27.56,83,76,10.92,MH,1701927766
2,2,whitehorse,60.7161,-135.0538,-1.57,80,100,6.69,CA,1701927767
3,3,puerto natales,-51.7236,-72.4875,3.64,94,2,2.02,CL,1701927768
4,4,toledo,39.8333,-4.0000,6.84,81,99,1.60,ES,1701927769


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = (city_data_df[(city_data_df["Max Temp"] < 23.889)])
ideal_df = ideal_df[ideal_df["Humidity"] < 60]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,fada,9.5667,8.3000,19.89,20,21,2.79,NG,1701927774
25,25,kodiak,57.7900,-152.4072,2.73,52,0,7.20,US,1701927793
32,32,ghat,24.9647,10.1728,15.47,21,3,2.31,LY,1701927801
54,54,makhambet,47.6667,51.5833,-2.36,56,100,7.49,KZ,1701927824
58,58,hasaki,35.7333,140.8333,20.16,47,0,13.16,JP,1701927827


### Step 3: Create a new DataFrame called `hotel_df`.

In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy().reset_index()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,fada,NG,9.5667,8.3000,20,
1,kodiak,US,57.7900,-152.4072,52,
2,ghat,LY,24.9647,10.1728,21,
3,makhambet,KZ,47.6667,51.5833,56,
4,hasaki,JP,35.7333,140.8333,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [5]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fada - nearest hotel: No hotel found
kodiak - nearest hotel: Shelikof Lodge
ghat - nearest hotel: فندق تاسيلي
makhambet - nearest hotel: No hotel found
hasaki - nearest hotel: 大新旅館
al jawf - nearest hotel: No hotel found
mandalgovi - nearest hotel: Б
esfarayen - nearest hotel: هتل ساریگل
al bawiti - nearest hotel: Old Oasis Hotel
gorom-gorom - nearest hotel: No hotel found
tahoua - nearest hotel: Hotel Bungalow
uttarkashi - nearest hotel: Shivalinga
codrington - nearest hotel: No hotel found
gazli - nearest hotel: No hotel found
al mindak - nearest hotel: No hotel found
el bauga - nearest hotel: No hotel found
al muwayh - nearest hotel: فندق ومطعم الموية
jinchang - nearest hotel: No hotel found
ha'il - nearest hotel: فندق كلاسيك
blackmans bay - nearest hotel: Villa Howden
dongola - nearest hotel: Lord
mount gambier - nearest hotel: The Old Mount Gambier Gaol
markala - nearest hotel: No hotel found
qarazhal - nearest hotel: No hotel found
illizi - nearest hotel: بو

,City,Country,Lat,Lng,Humidity,Hotel Name
0,fada,NG,9.5667,8.3000,20,No hotel found
1,kodiak,US,57.7900,-152.4072,52,Shelikof Lodge
2,ghat,LY,24.9647,10.1728,21,فندق تاسيلي
3,makhambet,KZ,47.6667,51.5833,56,No hotel found
4,hasaki,JP,35.7333,140.8333,47,大新旅館
...,...,...,...,...,...,...
94,datong,CN,40.0936,113.2914,24,梧桐格客栈
95,djenne,ML,13.9061,-4.5533,20,Hotel Chez Baba
96,fountain,US,38.6822,-104.7008,27,Microtel Inn & Suites Fountain North
97,ushibukamachi,JP,32.1906,130.0228,48,Hotel Harbor Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "City"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)